<a href="https://colab.research.google.com/github/Renuka239/Deep-Learning-Project/blob/main/Pydantic_Dataset_Profiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I built a dataset ingestion and profiling layer using Pydantic schemas. Any uploaded CSV is converted into a DatasetSchema capturing column metadata, inferred data types, missingness, and statistics, along with a DataPreview for downstream analytics, visualization, and LLM interaction.

Cell 1: Install packages
-q just means “quiet” (less output).

In [ ]:
!pip -q install pandas pydantic numpy


Cell 2: Imports
typing → for type hints in Pydantic models
datetime → timestamp when schema is created
io → read uploaded file bytes as a file-like object
pandas → read and manipulate CSV
numpy → handle NaN and numeric conversions
Pydantic → define schemas (your “key data structures”)
Colab files → browser-based file upload

In [ ]:
from __future__ import annotations

from typing import Any, Dict, List, Optional, Literal
from datetime import datetime
import io

import pandas as pd
import numpy as np
from pydantic import BaseModel, Field, ConfigDict

from google.colab import files


Cell 3: Pydantic Schemas (KEY DATA STRUCTURES)
1)DataType-This restricts column types to known values.
So no random strings like "int64" or "object"
2)ColumnSchema-Represents ONE column in the dataset.Each column stores:
name → column name
dtype → inferred logical type
nullable → does it contain missing values?
unique → does it look like an ID?
Statistics:non-null count
null count + percentage
distinct values
min / max / mean (only for numeric columns)
This is your column-level metadata.

3)DatasetSchema-Represents the ENTIRE dataset.It stores:dataset name (uploaded filename),timestamp (when schema was created),number of rows,number of column,list of ColumnSchema
Think of this as a data catalog entry.

4)DataPreview-A safe sample of the dataset.Why it exists:You don’t want to send full data to UI/LLMs. You want just a few rows, JSON-safe.It stores:dataset name,sample size, first few rows as dictionaries

In [ ]:
DataType = Literal["string", "integer", "number", "boolean", "datetime", "category", "unknown"]

class ColumnSchema(BaseModel):
    model_config = ConfigDict(extra="forbid")

    name: str
    dtype: DataType
    nullable: bool
    unique: bool

    non_null_count: int
    null_count: int
    null_pct: float

    distinct_count: int
    min: Optional[float] = None
    max: Optional[float] = None
    mean: Optional[float] = None


class DatasetSchema(BaseModel):
    model_config = ConfigDict(extra="forbid")

    dataset_name: str
    created_at: datetime = Field(default_factory=datetime.utcnow)

    row_count: int
    column_count: int
    columns: List[ColumnSchema]


class DataPreview(BaseModel):
    model_config = ConfigDict(extra="forbid")

    dataset_name: str
    sample_size: int
    rows: List[Dict[str, Any]]


Cell 4: Helper function (data type inference)This function decides:

“What kind of data is this column logically?”

Checks happen in order:Integer → "integer",Float → "number",Boolean → "boolean",Datetime → "datetime",Low-cardinality → "category",Otherwise → "string"
This makes the code dataset-agnostic.

In [ ]:
def infer_dtype(series: pd.Series) -> DataType:
    if pd.api.types.is_integer_dtype(series):
        return "integer"
    if pd.api.types.is_float_dtype(series):
        return "number"
    if pd.api.types.is_bool_dtype(series):
        return "boolean"
    if pd.api.types.is_datetime64_any_dtype(series):
        return "datetime"
    if series.nunique(dropna=True) < 20:
        return "category"
    return "string"


Cell 5: Upload CSV (Colab browser upload)-This opens a browser upload dialog.
What’s happening:It gives you the file as bytes,io.BytesIO converts bytes → file-like object,Pandas reads it like a normal CSV
Now df is a DataFrame.

In [ ]:
uploaded = files.upload()  # choose a CSV file in the browser

if len(uploaded) == 0:
    raise ValueError("No file uploaded. Please upload a CSV file.")

filename = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[filename]), encoding='latin1')

print("Loaded file:", filename)
df.head()

Saving diabetes.csv to diabetes (2).csv
Loaded file: diabetes (2).csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Cell 6: Build DatasetSchema (core logic)-This is where raw data → structured schema happens.For each column:Infer data type,Count missing values,Compute missing percentage,Count distinct values,Check uniqueness
If numeric → compute min, max, mean
All these values are passed into ColumnSchema.
Each column schema is appended to a list.
Create DatasetSchema-This bundles:filename,row count,column count,all column schemas
Now you have one validated object representing the dataset.

In [ ]:
columns = []
rows = len(df)

for col in df.columns:
    s = df[col]
    dtype = infer_dtype(s)

    null_count = int(s.isna().sum())
    non_null_count = rows - null_count
    null_pct = round((null_count / rows) * 100, 2)

    distinct_count = int(s.nunique(dropna=True))
    unique = distinct_count == non_null_count

    min_v = max_v = mean_v = None
    if dtype in ["integer", "number"]:
        # numeric stats (safe conversion)
        numeric = pd.to_numeric(s, errors="coerce")
        if numeric.notna().any():
            min_v = float(numeric.min())
            max_v = float(numeric.max())
            mean_v = float(numeric.mean())

    columns.append(
        ColumnSchema(
            name=str(col),
            dtype=dtype,
            nullable=null_count > 0,
            unique=bool(unique),
            non_null_count=non_null_count,
            null_count=null_count,
            null_pct=null_pct,
            distinct_count=distinct_count,
            min=min_v,
            max=max_v,
            mean=mean_v,
        )
    )

dataset_schema = DatasetSchema(
    dataset_name=filename,
    row_count=rows,
    column_count=len(df.columns),
    columns=columns
)

dataset_schema


DatasetSchema(dataset_name='diabetes (2).csv', created_at=datetime.datetime(2026, 2, 7, 0, 25, 51, 632411), row_count=768, column_count=9, columns=[ColumnSchema(name='Pregnancies', dtype='integer', nullable=False, unique=False, non_null_count=768, null_count=0, null_pct=0.0, distinct_count=17, min=0.0, max=17.0, mean=3.8450520833333335), ColumnSchema(name='Glucose', dtype='integer', nullable=False, unique=False, non_null_count=768, null_count=0, null_pct=0.0, distinct_count=136, min=0.0, max=199.0, mean=120.89453125), ColumnSchema(name='BloodPressure', dtype='integer', nullable=False, unique=False, non_null_count=768, null_count=0, null_pct=0.0, distinct_count=47, min=0.0, max=122.0, mean=69.10546875), ColumnSchema(name='SkinThickness', dtype='integer', nullable=False, unique=False, non_null_count=768, null_count=0, null_pct=0.0, distinct_count=51, min=0.0, max=99.0, mean=20.536458333333332), ColumnSchema(name='Insulin', dtype='integer', nullable=False, unique=False, non_null_count=768

Cell 7: Create DataPreview-Key things here:Takes first 5 rows,Converts NaN → None (JSON-safe),Converts rows into list of dictionaries
This is perfect for:UI display,LLM prompts,Debugging,Logs

In [ ]:
preview = DataPreview(
    dataset_name=filename,
    sample_size=5,
    rows=df.head(5).replace({np.nan: None}).to_dict(orient="records"),
)

preview


DataPreview(dataset_name='OnlineRetail (2).csv', sample_size=5, rows=[{'InvoiceNo': '536365', 'StockCode': '85123A', 'Description': 'WHITE HANGING HEART T-LIGHT HOLDER', 'Quantity': 6, 'InvoiceDate': '12/1/2010 8:26', 'UnitPrice': 2.55, 'CustomerID': 17850.0, 'Country': 'United Kingdom'}, {'InvoiceNo': '536365', 'StockCode': '71053', 'Description': 'WHITE METAL LANTERN', 'Quantity': 6, 'InvoiceDate': '12/1/2010 8:26', 'UnitPrice': 3.39, 'CustomerID': 17850.0, 'Country': 'United Kingdom'}, {'InvoiceNo': '536365', 'StockCode': '84406B', 'Description': 'CREAM CUPID HEARTS COAT HANGER', 'Quantity': 8, 'InvoiceDate': '12/1/2010 8:26', 'UnitPrice': 2.75, 'CustomerID': 17850.0, 'Country': 'United Kingdom'}, {'InvoiceNo': '536365', 'StockCode': '84029G', 'Description': 'KNITTED UNION FLAG HOT WATER BOTTLE', 'Quantity': 6, 'InvoiceDate': '12/1/2010 8:26', 'UnitPrice': 3.39, 'CustomerID': 17850.0, 'Country': 'United Kingdom'}, {'InvoiceNo': '536365', 'StockCode': '84029E', 'Description': 'RED WO